# Whitebox Workflows (WbW) for Geomorphometry

## Introduction

This tutorial is intended to demonstrate the use of Whitebox Workflows for Python (WbW) for performing a geomorphometric analysis. It accompanies the book chapter: 

Lindsay, J.B. (In Review) "Chapter 17 Geomorphometry in Whitebox", in Rueter, H., Grohman, C. and Lecours, V. (eds.) *Geomorphometry: Concepts, Software, Applications (2nd edition)*, Elsevier.

Geomorphometry is the field focused on understanding landscape processes using digital topography, i.e., digital elevation models (DEMs). This tutorial will not cover all of the functionality related to geomorphometry contained within WbW. For more information, you may refer to the [user manual](https://www.whiteboxgeo.com/manual/wbw-user-manual/book/preface.html). 

## Setting up WbW

If you haven't already done so, install the Whitebox Worfklows for Python pip package.

In [ ]:
pip install whitebox-workflows

Or if you already have it installed in your Jupyter environment and need to update to the latest version...

In [ ]:
pip install whitebox-workflows -U

We need to import the `WbEnvironment` class from the `whitebox_workflows` library into our script.

In [ ]:
from whitebox_workflows import download_sample_data, show, WbEnvironment, WbPalette
import matplotlib.pyplot as plt

wbe = WbEnvironment()
wbe.verbose = True # Let each of the function calls output to stdout.

print(wbe.version()) # Let's see what version of WbW we're working with

## Working with DEM data

Let's create a new script to download some sample data. Here we'll grab the Ponui Island lidar DTM. The script below will download the data for us, assign the directory to which these data are downloaded to the `WbEnvironment` working directory and lastly print this location so we can know where the data are being stored. Notice that it may take a few minutes to download the data. In the event that the download takes more than a few minutes, the connection may timeout and you will receive an error. If this should happen, you may download the dataset directly [from here](http://www.whiteboxgeo.com/sample_data/Ponui_DTM.zip) but you will need to update the `wbe.working_directory` to the folder containing the downloaded data.

In [ ]:
# Download a sample dataset and set the working directory to the location of these data
wbe.working_directory = download_sample_data('Ponui_DTM')
print(f'Data have been stored in: {wbe.working_directory}')

Now let's read in the DEM file and generate a multidirectional hillshade image for visualization.

In [ ]:
# Read in the DEM file.
dtm = wbe.read_raster('DTM.tif') # This DEM file is contained in the downloaded data folder.

# Let's print some metadata for this file.
print(f"Rows: {dtm.configs.rows}")
print(f"Columns: {dtm.configs.columns}")
print(f"Min X: {dtm.configs.west}")
print(f"Max X: {dtm.configs.east}")
print(f"Min Y: {dtm.configs.south}")
print(f"Max Y: {dtm.configs.north}")
print(f"Resolution (x direction): {dtm.configs.resolution_x}")
print(f"Resolution (y direction): {dtm.configs.resolution_y}")
print(f"NoData value: {dtm.configs.nodata}")
dtm.update_min_max() # Find the raster min/max values
print(f"Min. value: {dtm.configs.minimum}")
print(f"Max. value: {dtm.configs.maximum}")
print(f"Data type: {dtm.configs.data_type}")

# Now for a simple visualization of the DTM...
show(dtm, skip=2, colorbar_kwargs={'label': 'Elevation (m)'})

Let's create a multi-directional hillshade and use it to visualize the DTM with some hypsometric tinting.

In [ ]:
# create a hillshade image for visualization.
wbe.verbose = False # No need for progress updates
hillshade = wbe.multidirectional_hillshade(dtm, full_360_mode=False)
wbe.write_raster(hillshade, 'hillshade.tif', compress=True)

# Let's visualize the DEM.
fig, ax = plt.subplots()
ax = show(
    dtm, 
    ax=ax, 
    title='Ponui Island', 
    cmap=WbPalette.Earthtones, 
    figsize=(8,7), 
    skip=2, 
    colorbar_kwargs={'label': 'Elevation (m)', 'location': "right", 'shrink': 0.5}
)
ax = show(hillshade, ax=ax, cmap='grey', clip_percent=10.0, skip=2, alpha=0.15, zorder=2)

plt.show()

Let's smooth the surface roughness in the DTM using the feature preserving smoothing method of Lindsay et al. (2019).

In [ ]:
# Perform the smoothing
dtm_smoothed = wbe.feature_preserving_smoothing(
    dem = dtm, 
    filter_size = 15, 
    normal_diff_threshold = 25.0, 
    iterations = 25,
    max_elevation_diff = 4.0
)

# Now let's visualize the difference. First generate a comparison hillshade...
hillshade_smoothed = wbe.multidirectional_hillshade(dtm_smoothed, full_360_mode=False)
wbe.write_raster(hillshade_smoothed, 'hillshade_smoothed.tif', compress=True)

# Now plot both raw and smoothed hillshade rasters.
fig, ax = plt.subplots(2, 1, figsize=(5, 10))
fig.tight_layout()

ax[0] = show(
    hillshade, 
    ax=ax[0], 
    title={'label': 'Original Hillshade', 'fontsize': 10, 'fontweight': 'bold'},
    cmap='grey', 
    clip_percent=1.0, 
    skip=2
)
ax[0].tick_params(axis='both', labelsize=7)
ax[0].xaxis.get_offset_text().set_fontsize(7)
ax[0].yaxis.get_offset_text().set_fontsize(7)

ax[1] = show(
    hillshade_smoothed, 
    ax=ax[1], 
    title={'label': 'FPS Hillshade', 'fontsize': 10, 'fontweight': 'bold'}, 
    cmap='grey', 
    clip_percent=1.0, 
    skip=2
)
ax[1].tick_params(axis='both', labelsize=7)
ax[1].xaxis.get_offset_text().set_fontsize(7)
ax[1].yaxis.get_offset_text().set_fontsize(7)

# Zoom to a smaller area so we can see the difference.
ax[0].set_xlim([1794000.0, 1795000.0])
ax[0].set_ylim([5918000.0, 5919000.0])
ax[1].set_xlim([1794000.0, 1795000.0])
ax[1].set_ylim([5918000.0, 5919000.0])

plt.show()

## Extracting land-surface parameters (LSPs)

Now let's extract some common land-surface parameters (LSPs), the basic building blocks of a geomorphometric analysis.

In [ ]:
# Slope and aspect are two of the most common LSPs. Notice that we're combining the writing of the
# output raster and the running of the function in one line. If you don't need to reuse the raster 
# objects created by a function and are only saving it to file this makes sense.
wbe.write_raster(wbe.slope(dtm, units="degrees"), 'slope.tif', compress=True)
wbe.write_raster(wbe.aspect(dtm), 'aspect.tif', compress=True)

# Surface curvatures describe surface shape. Note curvatures frequently display wide dynamic ranges.
# To avoid loss of information on the spatial distribution of their values in mapping, a logarithmic 
# transform can be applied using the approach of Shary et al. (2002).
wbe.write_raster(wbe.profile_curvature(dtm, log_transform=True), 'prof_curv.tif', compress=True)
wbe.write_raster(wbe.tangential_curvature(dtm, log_transform=True), 'tan_curv.tif', compress=True)
wbe.write_raster(wbe.plan_curvature(dtm, log_transform=True), 'plan_curv.tif', compress=True)
wbe.write_raster(wbe.maximal_curvature(dtm, log_transform=True), 'max_curv.tif', compress=True)
wbe.write_raster(wbe.mean_curvature(dtm, log_transform=True), 'mean_curv.tif', compress=True)
wbe.write_raster(wbe.gaussian_curvature(dtm, log_transform=True), 'gauss_curv.tif', compress=True)
wbe.write_raster(wbe.total_curvature(dtm, log_transform=True), 'total_curv.tif', compress=True)

# Let's display minimal curvature.
min_curv = wbe.minimal_curvature(dtm, log_transform=True).clamp(-4.0, 4.0)
wbe.write_raster(min_curv, 'min_curv.tif', compress=True)

# Now plot the data.
fig, ax = plt.subplots()

ax = show(
    min_curv, 
    ax=ax, 
    title={'label': 'Minimal Curvature', 'fontsize': 10, 'fontweight': 'bold'},
    cmap=WbPalette.BlueYellowRed, 
    clip_percent=0.0, 
    colorbar_kwargs={'label': 'Ln(m$\mathregular{^{-1}}$)', 'location': "right", 'shrink': 0.5},
    skip=2
)

# Zoom to a smaller area so we can see the difference.
ax.set_xlim([1794000.0, 1795000.0])
ax.set_ylim([5918000.0, 5919000.0])

plt.show()

The following LSPs can be used to characterize surface roughness and complexity.

In [8]:
wbe.write_raster(wbe.circular_variance_of_aspect(dtm, filter_size = 21), 'circular_variance_of_aspect.tif', compress=True)
wbe.write_raster(wbe.edge_density(dtm, filter_size=21, normal_diff_threshold=5.0), 'edge_density.tif', compress=True)
wbe.write_raster(wbe.spherical_std_dev_of_normals(dtm, filter_size = 21), 'spherical_sd_norms.tif', compress=True)
wbe.write_raster(wbe.standard_deviation_of_slope(dtm, filter_size = 21), 'stdev_slope.tif', compress=True)
wbe.write_raster(wbe.surface_area_ratio(dtm), 'surface_area_ratio.tif', compress=True)
wbe.write_raster(wbe.ruggedness_index(dtm), 'ruggedness_index.tif', compress=True)

Most of the measures of surface roughness and complexity above are measured for local neighbourhoods of a specified size (`filter_size = 21`). You may modify the `filter_size` parameter to see the impact of changing the scale of analysis on the output spatial distributions. Note that the grid resolution of the DEM is 1 m. Later we'll explore an alternative method for evaluating the multiscale nature of LSPs.

Measures of local topographic position assess how elevated or low-lying a site is relative to its neighbouring landscape.

In [ ]:
wbe.write_raster(wbe.deviation_from_mean_elevation(dtm, filter_size_x=21, filter_size_y=21), 'dev.tif', compress=True)
wbe.write_raster(wbe.difference_from_mean_elevation(dtm, filter_size_x=21, filter_size_y=21), 'diff.tif', compress=True)
wbe.write_raster(wbe.elevation_percentile(dtm, filter_size_x=21, filter_size_y=21), 'ep.tif', compress=True)
wbe.write_raster(wbe.percent_elev_range(dtm, filter_size_x=21, filter_size_y=21), 'percent_elev_range.tif', compress=True)

## Multiscale geomorphometric analysis

Many of the LSPs calculated above are scale dependent and require us to specify a measurement scale, usually in the form of a `filter_size` parameter. This parameter allows us to calculate these parameters at a single, uniform spatial scale. For many LSPs, Whitebox also allows us to calculate *scale mosiacs*. An LSP scale mosaic are calculated by estimating the normalized value of the LSP across a range of spatial scales, known as a scale space or stack, and then to identify the scale at which each grid cell is most expressive. This is known as the characteristic scale or key scale and it can be different for different locations. An LSP scale mosaic therefore represents the value of that LSP at the key scale of each individual grid cell. In comparison to the uniform scale approach, scale mosaics provide locally scale optimized representations of the LSP being measured. 

Let's create scale mosaics of deviation from mean elevation (DEV), a measure of relative topographic position, at three broadly defined scale ranges, including local, intermediate (meso), and broad scale ranges.

In [ ]:
# Let's turn off the verbose mode because these tools are pretty chatty
wbe.verbose = False

# Note, this processing may take a long while to complete.

# Start with a local scale range of 0 - 150 m.
print('Calculating the local scale range...')
dev_local, key_scales_local = wbe.max_elevation_deviation(dtm, min_scale=1, max_scale=150, step_size=1)
wbe.write_raster(dev_local, 'dev_multiscale_local.tif')
wbe.write_raster(key_scales_local, 'key_scales_local.tif')

# Now plot the data.
fig, ax = plt.subplots(1, 2, figsize=(9.5, 8.0))
fig.tight_layout()

ax[0] = show(
    dev_local.clamp(-2.5, 2.5), 
    ax=ax[0], 
    title={'label': 'Local Range DEVmax', 'fontsize': 10, 'fontweight': 'bold'},
    cmap=WbPalette.BlueYellowRed,
    colorbar_kwargs={'label': 'DEVmax', 'location': "right", 'shrink': 0.25}, 
    skip=2
)
ax[0].tick_params(axis='both', labelsize=7)
ax[0].xaxis.get_offset_text().set_fontsize(7)
ax[0].yaxis.get_offset_text().set_fontsize(7)
# Let's zoom in on this one because of the fine detail.
ax[0].set_xlim([1794000.0, 1795000.0])
ax[0].set_ylim([5918000.0, 5919000.0])

ax[1] = show(
    key_scales_local, 
    ax=ax[1], 
    title={'label': 'Key Scales', 'fontsize': 10, 'fontweight': 'bold'}, 
    cmap=WbPalette.BlueGreenYellow,
    colorbar_kwargs={'label': 'Key Scale', 'location': "right", 'shrink': 0.25},
    skip=2
)
ax[1].tick_params(axis='both', labelsize=7)
ax[1].xaxis.get_offset_text().set_fontsize(7)
ax[1].yaxis.get_offset_text().set_fontsize(7)
ax[1].set_xlim([1794000.0, 1795000.0])
ax[1].set_ylim([5918000.0, 5919000.0])

plt.show()


The DEVmax scale mosaic above contains more topographic information than the `dev.tif` raster that we generated in the previous section using a single uniform scale of 21 m. Notice that in addition to the scale mosaic, each of the multiscale LSP functions in Whitebox also output a second raster for the key scales (i.e. the characteristic scale at which the 'optimal' LSP value is calculated). This raster tells you what scale each grid cell in the scale mosaic was calculated at and can contain useful information in its own right.

In [ ]:
# Note, this processing may take a long while to complete.

# Now model the local scale range of 150 - 500 m.
print('Calculating the intermediate scale range...')
dev_meso, key_scales_meso = wbe.max_elevation_deviation(dtm, min_scale=150, max_scale=500, step_size=2)
wbe.write_raster(dev_meso, 'dev_multiscale_meso.tif')
wbe.write_raster(key_scales_meso, 'key_scales_meso.tif')

# Now plot the data.
fig, ax = plt.subplots(1, 2, figsize=(9.5, 8.0))
fig.tight_layout()

ax[0] = show(
    dev_meso.clamp(-2.5, 2.5), 
    ax=ax[0], 
    title={'label': 'Meso Range DEVmax', 'fontsize': 10, 'fontweight': 'bold'},
    cmap=WbPalette.BlueYellowRed,
    colorbar_kwargs={'label': 'DEVmax', 'location': "right", 'shrink': 0.25}, 
    skip=2
)
ax[0].tick_params(axis='both', labelsize=7)
ax[0].xaxis.get_offset_text().set_fontsize(7)
ax[0].yaxis.get_offset_text().set_fontsize(7)

ax[1] = show(
    key_scales_meso, 
    ax=ax[1], 
    title={'label': 'Key Scales', 'fontsize': 10, 'fontweight': 'bold'}, 
    cmap=WbPalette.BlueGreenYellow,
    colorbar_kwargs={'label': 'Key Scale', 'location': "right", 'shrink': 0.25},
    skip=2
)
ax[1].tick_params(axis='both', labelsize=7)
ax[1].xaxis.get_offset_text().set_fontsize(7)
ax[1].yaxis.get_offset_text().set_fontsize(7)

plt.show()

In [ ]:
# Note, this processing may take a long while to complete.

# Finally, model a broad scale range of 500 - 2000 m.
print('Calculating the broad scale range...')
dev_broad, key_scales_broad = wbe.max_elevation_deviation(dtm, min_scale=500, max_scale=2000, step_size=10)
wbe.write_raster(dev_broad, 'dev_multiscale_broad.tif')
wbe.write_raster(key_scales_broad, 'key_scales_broad.tif')

# Now plot the data.
fig, ax = plt.subplots(1, 2, figsize=(9.5, 8.0))
fig.tight_layout()

ax[0] = show(
    dev_broad.clamp(-2.5, 2.5), 
    ax=ax[0], 
    title={'label': 'Broad Range DEVmax', 'fontsize': 10, 'fontweight': 'bold'},
    cmap=WbPalette.BlueYellowRed,
    colorbar_kwargs={'label': 'DEVmax', 'location': "right", 'shrink': 0.25}, 
    skip=2
)
ax[0].tick_params(axis='both', labelsize=7)
ax[0].xaxis.get_offset_text().set_fontsize(7)
ax[0].yaxis.get_offset_text().set_fontsize(7)

ax[1] = show(
    key_scales_broad, 
    ax=ax[1], 
    title={'label': 'Key Scales', 'fontsize': 10, 'fontweight': 'bold'}, 
    cmap=WbPalette.BlueGreenYellow,
    colorbar_kwargs={'label': 'Key Scale', 'location': "right", 'shrink': 0.25},
    skip=2
)
ax[1].tick_params(axis='both', labelsize=7)
ax[1].xaxis.get_offset_text().set_fontsize(7)
ax[1].yaxis.get_offset_text().set_fontsize(7)

plt.show()

When you create three scale mosaics for broadly defined local, intermediate, and broad scale ranges, as we have here, it is possible to join each of the three scale mosaics into a single multiscale topographic position (MSTP) image. The MSTP is a colour composite that is really only meant for visualization purposes, but can be very effective for interpreting landscapes. In effect, with a MSTP image, you are using colour to represent spatial scales. Pixels that are blue are most deviated (either elevated or low-lying) at the local scale, green pixels are most deviated at the intermediate scale, and redish pixels are most deviated at the broadest tested scale range. Of course, you can also have combinations therein, e.g. a yellow pixel is one that is deviated at the intermediate and broad scales, but not particularly deviated at the local scale.

While the MSTP image is useful for visualization and interpretation, ultimately it is the scale mosaics (DEVmax above) images that are most useful as modelling inputs (i.e., predictors).

In [ ]:
print('Calculating the multiscale topographic position image...')
mstp = wbe.multiscale_topographic_position_image(dev_local, dev_meso, dev_broad)
wbe.write_raster(mstp, 'mstp.tif')

# Now plot the data.
fig, ax = plt.subplots(2, 1, figsize=(5, 10))
fig.tight_layout()

ax[0] = show(
    mstp, 
    ax=ax[0], 
    title={'label': 'MSTP Image', 'fontsize': 10, 'fontweight': 'bold'},
    clip_percent=1.0, 
    skip=2,
    zorder=1
)
ax[0] = show(hillshade, ax=ax[0], cmap='grey', clip_percent=10.0, skip=2, alpha=0.15, zorder=2)
ax[0].tick_params(axis='both', labelsize=7)
ax[0].xaxis.get_offset_text().set_fontsize(7)
ax[0].yaxis.get_offset_text().set_fontsize(7)

ax[1] = show(
    mstp, 
    ax=ax[1], 
    title={'label': 'MSTP Inset Image', 'fontsize': 10, 'fontweight': 'bold'}, 
    clip_percent=1.0, 
    skip=2,
    zorder=1
)
ax[1] = show(hillshade, ax=ax[1], cmap='grey', clip_percent=10.0, skip=2, alpha=0.15, zorder=2)
ax[1].tick_params(axis='both', labelsize=7)
ax[1].xaxis.get_offset_text().set_fontsize(7)
ax[1].yaxis.get_offset_text().set_fontsize(7)

# Zoom to a smaller area so we can see the difference.
ax[1].set_xlim([1794000.0, 1795000.0])
ax[1].set_ylim([5918000.0, 5919000.0])

plt.show()

Whitebox contains functions for calculating other multiscale LSPs as well, including `multiscale_elevation_percentile`, `multiscale_roughness`, `multiscale_std_dev_normals`, `max_anisotropy_dev`, `gaussian_scale_space`, and `multiscale_curvatures`. Notice that  use of the `multiscale_curvatures` function **requires a WbW-Pro license**. This is how you might use the `multiscale_curvatures` tool to extract a multiscale version of minimal curvature for a scale range of 2-50 m.

In [26]:
curv_mosaic, key_scales = wbe.multiscale_curvatures(
    dtm, 
    curv_type = 'MinimalCurvature', 
    min_scale = 2, 
    step_size = 1, 
    num_steps = 49, 
    step_nonlinearity = 1.0,
    log_transform = False,
    standardize = True
  )
wbe.write_raster(curv_mosaic, 'ms_min_curv.tif')
wbe.write_raster(key_scales, 'ms_min_curv_key_scales.tif')